Notebook pour calculer les Excess Waiting Time.
Tout d'abord on a besoin des packages pour Cluster

In [1]:
import gtfs_functions as gtfs
import matplotlib
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import random
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# cluster visualizer
#%matplotlib inline
#from yellowbrick.cluster import KElbowVisualizer 

# sklearn kmeans
from sklearn.metrics.cluster import contingency_matrix

# pyclustering kmeans
from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import distance_metric
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.cluster.encoder import type_encoding
from pyclustering.cluster.encoder import cluster_encoder
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from numpy import unique
from numpy import where
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture

import csv

In [3]:
#We import the schedule

path_gtfs ='C:/Users/Maison/Documents/INFO-H423/Projet Data Mining/GTFS3sep/gtfs3sept.zip'

routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(path_gtfs,busiest_date=False)

In [4]:
#We import the calendar by hand 

calendardf = pd.read_table('calendar.txt',sep=',')          #export the calendar file into data frame
calendarDatesdf = pd.read_table('calendar_dates.txt',sep=',')  #also export the exceptions

In [5]:
def FromDayToService(date):
    '''Determine all the service_id concerned by a date, given in the format date=('20210907','tuesday')
        it first take a look at every regular service_id then examine the exceptions. 
        Returns a list of service_id of type string'''

    ListServiceId=[]
    for i in range(len(calendardf)):
        startdate,endate=calendardf.iloc[i,[8,9]]
        if int(date[0]) in range(int(startdate),int(endate)+1):
            if calendardf.loc[i,date[1]]==1:
                #It appends every service_id which contains the date, and for which there is a '1'
                # under right day in the week
                ListServiceId.append(str(calendardf.iloc[i,0]))     

    #Now we have to take a look a the exceptions
    #We selections all the exceptions in calendarDatesdf that concerns the date in the input
    ModifServiceId= calendarDatesdf[calendarDatesdf['date']==int(date[0])][['service_id','exception_type']]
    #print('Number of service_id involved in modifications: '+ str(len(ModifServiceId)))
    #for every modif we check the kind of exception
    for modif in ModifServiceId.iterrows():  
        service = str(modif[1]['service_id'])
        exception = modif[1]['exception_type']
        if exception == 1:
            #if the exception is 1, it means that we should change to a 1 the cell related to the date in
            # input, in the row equal to service in calendar
            ListServiceId.append(service)
            #print('ServiceId: '+str(service)+' correclty added' )
        elif (exception == 2) & (service in ListServiceId):
            #if the exception is 2, it means that we should change to a 0 the cell related to the date in
            # input, in the row equal to service in calendar
            #Since this service_id must have been selected in the first loop (in Calendar), we need to remove it
            ListServiceId.remove(service)
            #print('ServiceId: '+str(service)+' correclty removed' )
        elif (exception == 2) & (service not in ListServiceId):
            #if the exception is 2 but we haven't found the service id in the first loop, this is weird
            #In principle this should not happend, but we're never too carefull
            print('Warning : exception 2 but no service_id = ' +service+ ' found in calendar')
    return (ListServiceId)

def FromLineIdtoRouteId(LineId):
    
    '''Takes a lineid in argument a returns the correspondent routeid, since stop_times only 
        contains route_id'''

    routeId = routes[routes['route_short_name']==LineId]['route_id'].iloc[0]
    return routeId

def ScheduledTime(date,routeid,stopId):

    '''Takes a triplet (date,routeid,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. The schedule is composed as follows: if we are in a period of
        punctuality then returns the (arrivaltime,true), and if we are in regularity, 
        returns (TimeOfWaitFromPreviousVehicle,False), all in one list '''

    ListOfService = pd.Series(FromDayToService(date))  #List of service_id tha apply for that day
    #RouteId = FromLineIdtoRouteId(lineId)               #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==routeid)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    ArrivalTime = Scheduledf['arrival_time'].values
    #print('Number of service_id involved: '+ str(len(ListOfService)))
    #We compute every wainting time to determine punctuality of regularity (< or > 720 s)
    #Schedule = list(ArrivalTime[1:]-ArrivalTime[:-1])
    IntervalTime = list(ArrivalTime[1:]-ArrivalTime[:-1])
    #Schedule.insert(0,ArrivalTime[0])
    #True if punctuality, False if regularity
    Punctuality = np.array(IntervalTime)>720 
    #Ultimately, if we are a punctuality, we want the arrival time and if we are in regularity, we want the 
    #time bewteen two vehicles
    for i,punctuality in enumerate(Punctuality):
        if punctuality:
            if (0<i<(len(Punctuality)-1)):
                if ~(Punctuality[i-1]) & ~(Punctuality[i+1]):
                    Punctuality[i]=False
            else:
                IntervalTime[i]=ArrivalTime[i]
 
    #Schedule = list(zip(Schedule,Punctuality))
    #The format is a list of tuples
    if len(IntervalTime)!=len(ArrivalTime[:-1]):
        print('Attention: la longueur de Schedule et ArrivalTime ne match pas dans ScheduledTime ')
    return IntervalTime,Punctuality,ArrivalTime[:-1]

def RouteIdToLineId(routeId):

    '''Takes a routeid in argument a returns the correspondent lineid'''

    lineId = routes[routes['route_id']==routeId]['route_short_name'].iloc[0]
    return lineId

def TroncatedScheduleRegularity(date,routeid,stopId):

    '''Takes a triplet (date,routeid,stopId) in the format date=('20210907','tuesday') and returns the IntervalTime 
        and ArrivalTime cleared from the ponctuality point'''
    ScheduleNonCleared = ScheduledTime(date,routeid,stopId)
    TroncatedInterval = [intervaltime for i,intervaltime in enumerate(ScheduleNonCleared[0]) if not(ScheduleNonCleared[1][i])]
    TroncatedArrival = [arrivaltime for i,arrivaltime in enumerate(ScheduleNonCleared[2]) if not(ScheduleNonCleared[1][i])]
    if len(TroncatedInterval)!= len(TroncatedArrival):
        print('Attention: la longueur de TroncatedInterval et TroncatedArrival ne match pas dans TroncatedScheduleRegularity ')
    return TroncatedInterval,TroncatedArrival

Bellow is the cell that compute the middle stop from a line in a specific direction, not really relevant now

In [6]:
StopTimesGroupTrip = stop_times.groupby('trip_id')[['stop_sequence']].mean().round()
StopTimesGroupTrip['stop_sequence']=StopTimesGroupTrip['stop_sequence'].astype(int)

StopTimesOnlyMeanSeq = pd.merge(stop_times, StopTimesGroupTrip, how='right', on= ['trip_id','stop_sequence'])
StopTimesOnlyMeanSeq = StopTimesOnlyMeanSeq[['route_id','stop_sequence','stop_id','direction_id']]
DicOfMiddleStop={}

for route in StopTimesOnlyMeanSeq['route_id'].unique():
    StopTimeMeanSeqRoute = StopTimesOnlyMeanSeq[StopTimesOnlyMeanSeq['route_id']==route]
    directionIds = StopTimeMeanSeqRoute['direction_id'].unique()
    for direction in directionIds:
        #Ici je prends la première valeure que je trouve qui a la bonne route et direction
        # car il peut y avoir plusieurs candidat Middle stop pour chaque (ligne,direction)
        # ici je prend le premier parce que j'en ai très exactement rien à fouttre
        stop = StopTimeMeanSeqRoute[StopTimeMeanSeqRoute['direction_id']==direction]['stop_id'].values[0]
        value = str(route)+':'+str(direction)
        if DicOfMiddleStop.get(stop) is None:
            DicOfMiddleStop[stop]=value


#Format du dico : 'middle_stop_id': 'routeid:direction'    Ex: '5510': '41:0'

#Ici c'est juste pour lui dire quelle date prendre en exemple si on dit 'weekday', 'satuday', etc...
DicOfWeek = {'weekday':('20210915','wednesday'),'saturday':('20210918','saturday'),'sunday':('20210919','sunday')}

#liste des middle stop
ListOfMiddleStop= list(DicOfMiddleStop.keys())

def get_key(val):
    for key, value in DicOfMiddleStop.items():
        if val == value:
            return key


Now we need the algorithm to cluster 

In [7]:
# define dictionary for distance measures
distance_measures = {'euclidean': 0, 'squared euclidean': 1, 'manhattan': 2, 'chebyshev': 3, 
                    'canberra': 5, 'chi-square': 6}

# function defined to compute purity score using pyclustering for various distance measures
def Clustering(dist_measure,nclust,data):
    initial_centers = random_center_initializer(data.values, nclust, random_state=5).initialize()
    # instance created for respective distance metric
    instanceKm = kmeans(data.values, initial_centers=initial_centers, metric=distance_metric(dist_measure))
    # perform cluster analysis
    instanceKm.process()
    # cluster analysis results - clusters and centers
    pyClusters = instanceKm.get_clusters()
    pyCenters = instanceKm.get_centers()
    # enumerate encoding type to index labeling to get labels
    pyEncoding = instanceKm.get_cluster_encoding()
    pyEncoder = cluster_encoder(pyEncoding, pyClusters, data.values)
    pyLabels = pyEncoder.set_encoding(0).get_clusters()
    # function purity score is defined in previous section

    return(pyClusters,pyCenters)
    #return purity_score(IntervalTimeBetweenVehicle['TimeInterval'].values, pyLabels)

# print results
#for measure, value in distance_measures.items():
    #print(f"The purity score for {measure} distance is {round(pyPurity(value)*100, 2)}%")

Tout est en place pour commencer un exemple qui portera sur le mercredi 15/09

In [8]:
DateExample = ('20210915','wednesday')

D'abord on va venir mettre toute la data qui concerne cette date dans un CSV, par arrêt 

In [9]:
ListOfStopId  = list(stop_times['stop_id'].unique())
DicOfStopInRoute = {}
for route in routes['route_id']:
    ListOfStop = [stop for stop in stop_times[stop_times['route_id']==route]['stop_id'].unique()]
    DicOfStopInRoute[route]=ListOfStop

In [10]:
def ScheduledTimeCSV(date,routeid,stopId):

    '''Takes a triplet (date,lineId,stopId) with date in the format date=('20210907','tuesday')
        and returns the schedule for that triplet. THE DIFFERENCE WITH ScheduledTime IS THAT WE DONT
        NEED THE PUNCTUALITY'''

    ListOfService = pd.Series(FromDayToService(date))              #We need the routeid to seek in the GTFS
    #We select the relevant rows in GTFS stop_times
    Scheduledf = stop_times[(stop_times['stop_id']==stopId)&(stop_times['route_id']==routeid)&(stop_times['service_id'].isin(ListOfService))].sort_values('arrival_time')
    directionId = Scheduledf['direction_id'].unique()
    ArrivalTime = Scheduledf['arrival_time'].values
    

    return ArrivalTime,directionId

In [ ]:
#Runtime ~ 30min

#DO NOT RUN THE CELL, OTHERWISE IT WILL WRITE OVER THE EXISTING FILE AND YOU'LL HAVE TO WAIT 30min

headerSchedule = ['route_id','direction_id','stop_id','ArrivalTime']

with open('Schedule20210915.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(headerSchedule)
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            ArrivalTime,directionId = ScheduledTimeCSV(DateExample,route,stop)
            row = [route,directionId,stop]
            row.extend(ArrivalTime)
            #writer.writerow(row)
        k+=1
        if k==5:
            print(str(k*l)+' lignes encodées')
            l+=1
            k=0

Pour l'exemple regardons au format d'une ligne du CSV

In [12]:
ReadSchedule = csv.reader(open('Schedule20210915.csv', 'r'),delimiter=',')
for i,row in enumerate(ReadSchedule):
    if i==2:
        print(row)

['2', '[1 0]', '8161', '19800.0', '19939.0', '20433.0', '20700.0', '21053.0', '21764.0', '22293.0', '22533.0', '22893.0', '23106.0', '23496.0', '23742.0', '24126.0', '24401.0', '24746.0', '25007.0', '25366.0', '25627.0', '25986.0', '26247.0', '26606.0', '26867.0', '27226.0', '27487.0', '27846.0', '28107.0', '28466.0', '28727.0', '29086.0', '29347.0', '29706.0', '29967.0', '30326.0', '30587.0', '30946.0', '31207.0', '31566.0', '31827.0', '32186.0', '32447.0', '32806.0', '33067.0', '33426.0', '33687.0', '34046.0', '34307.0', '34666.0', '34927.0', '35286.0', '35547.0', '35906.0', '36167.0', '36526.0', '36787.0', '37215.0', '37407.0', '37846.0', '38021.0', '38628.0', '39142.0', '39248.0', '39592.0', '39868.0', '40492.0', '40667.0', '40942.0', '41117.0', '41392.0', '41582.0', '41842.0', '42017.0', '42292.0', '42467.0', '42742.0', '42917.0', '43192.0', '43367.0', '43642.0', '43817.0', '44092.0', '44267.0', '44542.0', '44717.0', '44992.0', '45167.0', '45442.0', '45617.0', '45892.0', '46067.0'

En premier lieu on a la route_id , puis la direction, puis le stop_id, puis toutes les arrivées classée par temps, toutes au format string


Ok maintenant qu'on a le schedule sur la journée qui nous intéresse, on peut commencer à calculer les TimeGroup via cluster, on calcul le SWT sur chaque(cfr slide STIB, je ne fais pas de moyenne mais je calcul sur tous les intervals)

In [ ]:
#Runtime ~ 35min 

#DO NOT RUN THE CELL, OTHERWISE IT WILL WRITE OVER THE EXISTING FILE AND YOU'LL HAVE TO WAIT 35min

header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

with open('TimeGroupWeekdayAllStopNewSWT.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for route in DicOfStopInRoute:
        for stop in DicOfStopInRoute[route]:
            DataTemp = TroncatedScheduleRegularity(('20210915','wednesday'),route,stop)
            IntervalTime = DataTemp[0]
            ArrivalTime = DataTemp[1]
            Data = pd.DataFrame({'ArrivalTime':ArrivalTime, 'TimeWaited':IntervalTime})
            if len(Data)>6:
                Clusters = Clustering(distance_measures['manhattan'],6,Data)[0]
                Clusters.sort(key=lambda x: x[0])
                row = [route,stop]
                for cluster in Clusters:
                    #MeanIntervalTimeInCluster = Data.iloc[cluster[0]:cluster[-1],1].mean()
                    SWT = Data.iloc[cluster[0]:cluster[-1],1].values
                    if sum(SWT)!=0:
                        SWT = sum(SWT**2)/(2*sum(SWT))
                    elif sum(SWT)==0:
                        print('Problem with SWT on stop: '+str)
                        SWT='Problem with SWT'
                    #rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(MeanIntervalTimeInCluster)
                    rangeCluster = str(Data.iloc[cluster[0],0])+'::'+str(Data.iloc[cluster[-1],0])+'::'+str(SWT)
                    row.append(rangeCluster)
                writer.writerow(row)
        k+=1
        if k==5:
            print(str(k*l)+' route encodées')
            l+=1
            k=0

Regardons au format d'une ligne de TimeGroupWeekdayAllStopNewSWT, que j'exporte en DataFrame

In [14]:
Clusterdf = pd.read_csv('TimeGroupWeekdayAllStopNewSWT.csv')
Clusterdf.head()

,route_id,stop_id,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5,Cluster6
0,2,8161,19800.0::32447.0::176.4866371471495,32806.0::43817.0::167.70143492870767,44092.0::53848.0::117.4270192701927,54167.0::63983.0::168.94111654441727,64378.0::74775.0::223.9188708281235,75296.0::86996.0::264.5808547008547
1,2,8151,19879.0::29475.0::182.8805752396832,29785.0::39221.0::164.92528613819414,39671.0::50021.0::225.0,50471.0::60737.0::175.89976621858563,61047.0::71249.0::161.03420897863165,72389.0::83175.0::299.6196921935843
2,2,8141,19950.0::29549.0::182.9182727367434,29859.0::39294.0::164.90943296237413,39744.0::50094.0::225.0,50544.0::60810.0::175.89976621858563,61120.0::71320.0::161.00303921568627,72460.0::83246.0::299.6196921935843
3,2,8131,20017.0::29621.0::182.98313202832153,29931.0::39363.0::164.86248939779475,39813.0::50163.0::225.0,50613.0::60881.0::175.92569146864042,61191.0::71387.0::160.9418399372303,72527.0::83313.0::299.6196921935843
4,2,8121,20098.0::29705.0::183.02045383574477,30015.0::39446.0::164.84704697274944,39896.0::50246.0::225.0,50696.0::60964.0::175.92569146864042,61274.0::71468.0::160.91181086913872,72608.0::83394.0::299.6196921935843


Format d'une cellule: on a une string avec 3 nombre séparés par des '::' . Les deux premiers sont les ranges de temps déterminé par le Cluster1, ici dans la première ligne c'est entre 19800 et 32447 secondes après minuit. Le dernier nombre est le SWT calculé dans ce cluster, via les intervales, voir slides STIB

Nickel ! Maintenant qu'on a ça il suffit de venir chercher les données JSON de Mehdi pour la même date, et de les trier par Cluster, c'est l'objet de la prochaine cellule

In [17]:
#Runtime ~ 10sec  

#DO NOT RUN THE CELL, OTHERWISE IT WILL WRITE OVER THE EXISTING FILE AND YOU'LL HAVE TO WAIT 10 sec

ReaderJson = csv.reader(open('stop_times_line_cleared2.csv', 'r'),delimiter=',')
TimeGroupClusterdf = pd.read_csv('TimeGroupWeekdayAllStopNewSWT.csv')
header = ['route_id','stop_id','Cluster1', 'Cluster2', 'Cluster3', 'Cluster4','Cluster5','Cluster6']

#Fonction pour lire le CSV JSON

with open('ClassifiedRegularityPointInJson.csv', 'w', encoding='UTF8',newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    # write the data
    k=0
    l=1
    for row in ReaderJson:
        if len(row)>2:
            route = int(FromLineIdtoRouteId(str((row[0]))))
            stop = str(row[1])
            RangeClusterInvolved = TimeGroupClusterdf[(TimeGroupClusterdf['route_id']==route) & (TimeGroupClusterdf['stop_id']==stop) ][['Cluster1','Cluster2','Cluster3','Cluster4','Cluster5','Cluster6']].values
            
            if len(RangeClusterInvolved)>0:
                RangeClusterInvolved = [RangeClust.split('::') for RangeClust in RangeClusterInvolved[0]]
                ArrivalTime = np.array(row[2].split(',')).astype(np.float)
                IntervalTime = ArrivalTime[1:]-ArrivalTime[:-1]
                RowToWrite=[route,stop]
                for rangecluster in RangeClusterInvolved:
                    borneinf = float(rangecluster[0])
                    bornesup = float(rangecluster[1])
                    swt = rangecluster[2]
                    DataInCluster= str(rangecluster[2])  #We start writing the row with SWT factor that we'll need later
                    for i,interval in enumerate(IntervalTime):
                        if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup):
                            DataInCluster+= ','+str(interval)
                    #DataInCluster = [IntervalTime[i] for i in range(len(IntervalTime))] #if (borneinf<ArrivalTime[i]) & (ArrivalTime[i]<=bornesup) ]
                    RowToWrite.append(DataInCluster)
                writer.writerow(RowToWrite)
            
        k+=1
        if k==500:
            print(str(k*l)+' arrêt encodées')
            l+=1
            k=0

500 arrêt encodées
1000 arrêt encodées
1500 arrêt encodées
2000 arrêt encodées
2500 arrêt encodées
3000 arrêt encodées
3500 arrêt encodées


Regardons encore une fois au format qu'on vient de créer

In [72]:
ReaderClassifiedJson = csv.reader(open('ClassifiedRegularityPointInJson.csv', 'r'),delimiter=',')
for i,row in enumerate(ReaderClassifiedJson):
    if row[1]=='6209':
        print(row[0])
        print(i)
    if i==1223:
        print(row[2])

5
94
10
101
15
1222
230.0,1229.141000032425,546.6260001659393,479.14299988746643,1936.6419999599457,30.618000030517578,32.46199989318848,30.926000118255615,699.8210000991821,31.436999797821045,442.4830000400543,31.847000122070312,64.7189998626709,63.592000007629395,31.23300004005432,31.13100004196167,94.10800004005432,0.0,33.382999897003174,31.64300012588501,32.46199989318848,32.66600012779236,411.66100001335144,32.56399989128113,980.7139999866486,158.00799989700317,379.6080000400543,633.0550000667572,32.15499997138977,31.028000116348267,415.9609999656677,316.1180000305176


Pour chaque ligne, on a une liste qui contient la route_id, le stop_id, puis 6 différentes string qui correspondent au 6 clusters et dans chaque string on a les intervals time du JSON qui sont contenus dans ce cluster, et qui sont séparés par une ','
ATTENTION: le premier éléments de chaque string de cluster correspond au SWT

On peut maintenant calculer les EWT !

In [73]:
def excesswaitingtime(swt,ArrivalTimeJSONArray):
    #both are given in second 
    #we give the EWT in second
    #ArrivalTimeJSONArray = np.array(ArrivalTimeJSONList)
    if swt == 'Problem with SWT':
        print('flag')
        return 'No EWT available'
    elif np.float(swt) ==np.float(0):
        return 'Problem with SWT = 0'
    elif sum(ArrivalTimeJSONArray)==0:
          return 'Problem with AWT = 0'
    else:
        awt = sum((ArrivalTimeJSONArray)**2)/(2*sum(ArrivalTimeJSONArray))
        ewt = awt-np.float(swt)
        return ewt
    

In [74]:
ReaderClassifiedJson = csv.reader(open('ClassifiedRegularityPointInJson.csv', 'r'),delimiter=',')

header = ['route_id','stop_id','EWT1', 'EWT2', 'EWT3', 'EWT4','EWT5','EWT6']


with open('EWTScore6clusterAllStop20210915.csv', 'w', encoding='UTF8',newline='') as file:
    writer = csv.writer(file)
    # write the header
    writer.writerow(header)
    # write the data
    
    for i,row in enumerate(ReaderClassifiedJson):
        route = row[0]
        stop = row[1]
        if i>0:
            RowToWrite=[route,stop]
            for IntervalInClusterReal in row[2:]:
                ListOfData = np.array(IntervalInClusterReal.split(','))
                swt = ListOfData[0]
                if (ListOfData[0] == 'Problem with SWT' ):
                    ewt = 'No EWT available'
                else:
                    ListOfData = ListOfData.astype(np.float)
                    ewt = excesswaitingtime(swt,ListOfData[1:])
                RowToWrite.append(ewt)
            writer.writerow(RowToWrite)

        if i%500==0:
            print(str(i)+' arrêts encodés')   
        
    

0 arrêts encodés
500 arrêts encodés
1000 arrêts encodés
1500 arrêts encodés
2000 arrêts encodés


In [77]:
EWTWednesday = pd.read_csv('EWTScore6clusterAllStop20210915.csv')
EWTWednesday.head(40)

,route_id,stop_id,EWT1,EWT2,EWT3,EWT4,EWT5,EWT6
0,2,8011,235.1704684871296,117.6303370171969,667.1613457127064,178.47164000154476,116.3278034028647,551.346912451316
1,2,8012,942.9780931428922,335.3570824869864,302.29227056869297,186.99103838115983,557.3219531239839,699.1541936616795
2,2,8021,560.5666639612948,496.29367262485306,620.6092103121475,155.14772767407794,757.7863127504505,958.486298733799
3,2,8022,518.9909108844051,326.5018847228581,393.2854247727356,455.9740270993774,275.4721475677577,312.41488599732907
4,2,8031,389.150885932425,321.0380122860721,493.94867596271456,278.08239659990403,508.38100467676384,675.4248682813193
5,2,8032,468.7692458017022,410.521387977245,892.0319025122567,395.0881988067657,581.8493374673272,538.5358367633348
6,2,8041,274.7566106878172,318.37840775444596,383.96877279510306,246.89441336304407,341.3303721401568,1095.8558158272035
7,2,8042,769.148892699684,387.13132897835357,655.236562660167,444.0263181422383,246.6649893018822,794.8373956042847
8,2,8051,357.4158095610145,649.4926721804679,410.2130720210465,629.7909777435246,305.26756441071666,444.86768926020267
9,2,8052,745.7452968762395,275.4747326938956,330.4712044236353,140.7687540905428,327.0134078640024,185.76947973936524
